<a href="https://colab.research.google.com/github/ruanrabbets-tech/NH3-Decomposition-Workflow/blob/main/Step_3_(Absorption_Energies).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ase==3.23.0

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# install quantum espresso from conda
!conda install conda-forge::qe

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Below imports necessary functions
from IPython.display import HTML, Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

from ase import Atoms
from ase.build import make_supercell, bulk, fcc111, molecule, add_adsorbate
from ase.io import read, write
from ase.visualize import view

import os
os.makedirs("output", exist_ok=True)

import numpy as np


# Below are functions necessary for viewing the structures
def view_x3d(atoms, idx=0):
    if isinstance(atoms[0], Atoms):
        # Assume this is a trajectory or struct list
        if (len(atoms) <= idx):
                print(f"The specified index exceeds the length of the trajectory. The length of the trajectory is {len(atoms)}.")
        return view(atoms[idx], viewer="x3d")
    else:
        return view(atoms, viewer="x3d")


def view_ase_atoms(atoms, rotation="0x,0y,0z", figsize=(4, 4), title="", scale=100):
    fig, ax = plt.subplots(figsize=figsize)
    write("output/tmp.png", atoms, rotation=rotation, scale=scale)
    img = mpimg.imread('output/tmp.png')
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
    plt.show()
    os.remove('output/tmp.png')
    return


def traj_to_apng(traj, rotation='30x,30y,30z'):
    imgs = []
    for atom in traj:
        supercell = make_supercell(atom, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        write('output/tmp.png', supercell, rotation=rotation, show_unit_cell=2)
        img = mpimg.imread('output/tmp.png')
        imgs.append(img)
    os.remove('output/tmp.png')

    fig, ax = plt.subplots()

    def update(frame):
        img = imgs[frame]
        ax.clear()
        ax.imshow(img)
        return []

    ani = FuncAnimation(fig, update, frames=len(imgs), blit=True)
    plt.close()
    return HTML(ani.to_jshtml())

### Below is a function I created for running quantum expresso.  I'm using my  **3rd version** because it saves the output straight to my google drive.

In [ ]:
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import BFGS
import shutil
import os

def run_qe_calculation(structure, calculation_type, pseudopotentials, calculation_label="Not_given"):
    """
    Runs a Quantum ESPRESSO calculation using ASE.

    Parameters:
        calculation_type (str): e.g., 'relax', 'scf', etc.
        pseudopotentials (dict): Mapping of elements to UPF files.
        structure (ase.Atoms): ASE Atoms object to be relaxed or calculated.

    Returns:
        float: Potential energy of the relaxed/final structure.
    """

    # Mount Google Drive ONLY if it's not already mounted
    from google.colab import drive
    try:
        drive.mount('/content/drive')
    except ValueError:
        pass  # Ignore if already mounted

    # Set the folder path on Google Drive (where I want my results to be saved)
    drive_folder = '/content/drive/MyDrive/My Results'  # Ruan and I adjust the folder path as needed
    calculation_folder = os.path.join(drive_folder, calculation_label)

    # Create the folder if it does not exist
    os.makedirs(calculation_folder, exist_ok=True)

    # Set up Quantum ESPRESSO input
    input_data = {
        'control': {
            'calculation': calculation_type,
            'verbosity': 'high',
            'pseudo_dir': os.path.abspath('./'),  # Use os.path.abspath to ensure absolute path
            'restart_mode': 'from_scratch',
            'tstress': True,
            'tprnfor': True,
            'outdir': calculation_folder  # Save QE native outputs here
        },
        'system': {
            'ecutwfc': 30,
            'ecutrho': 240,
            'occupations': 'smearing',
            'smearing': 'mv',               # Use smearing of "mp" or "mv" when dealing with metals and use "gaussian" when dealing with molecules.
            'degauss': 0.01,  # when working with adsorbate change to 0.001
            'nspin': 2
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_mode': 'plain',
            'mixing_beta': 0.7,
            # 'conv_thr': 1e-6
        }
    }

    profile = EspressoProfile(command='/usr/local/bin/pw.x', pseudo_dir='./')  # or os.path.abspath('./') if pseudo_dir is relative

    calc = Espresso(
        profile=profile,
        input_data=input_data,
        pseudopotentials=pseudopotentials,
        kpts=(2, 2, 1),               # ****************************************   Note from Prof. Cecil: if slab dimensions is (4,4,1) use k-point of "2,2,1", or use a gamma k-point i.e "1,1,1". We can use 4,4,4 for unit cells.
        directory=calculation_folder  # Save the output in the specified directory
    )

    structure.calc = calc

    # Run calculation
    energy = structure.get_potential_energy()

    #*************** Below code is an attempt to put all outputs into a specific folder *******########
    # Save energy summary to a text file
    output_file = os.path.join(calculation_folder, f'{calculation_label}_output.txt')
    with open(output_file, 'w') as f:
        f.write(f'Energy of the relaxed structure: {energy} eV\n')

    # Save final structure as CIF
    cif_path = os.path.join(calculation_folder, f'{calculation_label}_relaxed_structure.cif')
    write(cif_path, structure)  # Save relaxed structure

    # Redundantly ensure all generated files are in the Google Drive folder
    try:
        # Print source and destination to debug the issue
        print(f"Source directory: {calc.directory}")
        print(f"Destination directory: {calculation_folder}")

        # Copy files manually, avoid using copytree in case of special characters to avoid errors
        for item in os.listdir(calc.directory):
            s = os.path.join(calc.directory, item)
            d = os.path.join(calculation_folder, item)
            if os.path.isdir(s):
                shutil.copytree(s, d, dirs_exist_ok=True)  # If it's a directory, copy it recursively
            else:
                shutil.copy2(s, d)  # If it's a file, copy it

    except Exception as e:
        print(f"Error during file copy: {e}")

    return energy

### Below code generates a pseudopotential dictionary from all the available pseusopotential files available in the directory

In [ ]:
def generate_pseudopotential_dict(pseudo_dir='./'):
    """
    Automatically generates a pseudopotential dictionary by scanning a directory for .UPF files.

    Args:
        pseudo_dir (str): Path to the directory containing UPF files.

    Returns:
        dict: Dictionary in the form {element: filename}
    """
    pp_dict = {}
    for file in os.listdir(pseudo_dir):
        if file.endswith(".UPF"):
            element = file.split('.')[0]  # assumes element is the first part of the filename
            pp_dict[element] = file
    return pp_dict

### Below function I created helps me freeze atoms in the desired layers:

In [ ]:
def freeze(structure, no_of_layers_from_bottom_to_be_frozen):   #note "structure" refers to tht bulk structure in which I want to freeze some atoms.
  # Below gets the cordinate of the atoms in the structure
  atoms = structure
  import pandas as pd #importing this in case I fogot to import it earlier
  df = pd.DataFrame({
    "x": atoms.positions[:, 0],
    "y": atoms.positions[:, 1],
    "z": atoms.positions[:, 2],
    "symbol": atoms.symbols,
  })
  #Below isolates the z cordinates of all the atoms into a list
  z_list = df["z"].tolist()
  unique_z_values = list(set(z_list))  # removes duplicates
  sorted_z = sorted(unique_z_values, reverse=False)  # Sort from smallest to largest
  #identify the tresh of the z cordinates of the atoms to be frozen
  thresh =  (sorted_z[no_of_layers_from_bottom_to_be_frozen] + sorted_z[no_of_layers_from_bottom_to_be_frozen-1])/2
  # print(thresh) #************************************************************************************************************************************** For troubleshooting
  #below applies the constraint to the atoms in the layers I deliniated with the treshold
  from ase.constraints import FixAtoms   # In case I forget to call it earlier
  constraint = FixAtoms(mask=structure.positions[:, 2] < thresh)
  structure.set_constraint(constraint)
  return structure

# Calculation Steps

In [ ]:
#step 1: Upload the necessary Pseudopotential files
from google.colab import files
uploaded = files.upload()

In [ ]:
#Step 2: Generate a pseudo dictionary with my function
pseudo_dict = generate_pseudopotential_dict(pseudo_dir='./')
print(pseudo_dict)

{'Ni': 'Ni.pbesol-spn-kjpaw_psl.1.0.0.UPF', 'Zn': 'Zn.pbe-dnl-kjpaw_psl.1.0.0.UPF', 'Cu': 'Cu.pbe-dn-kjpaw_psl.1.0.0.UPF', 'Fe': 'Fe.pbesol-spn-kjpaw_psl.1.0.0.UPF', 'Co': 'Co.pbesol-spn-kjpaw_psl.0.3.1.UPF', 'V': 'V.pbesol-spnl-kjpaw_psl.1.0.0.UPF', 'Cr': 'Cr.pbesol-spn-kjpaw_psl.1.0.0.UPF', 'Sc': 'Sc.pbesol-spn-kjpaw_psl.1.0.0.UPF', 'Mn': 'Mn.pbesol-spn-kjpaw_psl.0.3.1.UPF', 'Ti': 'Ti.pbesol-spn-kjpaw_psl.1.0.0.UPF'}


### Below is a function to add an adsrobate to the ontop, bridge or hollow position.

In [ ]:
from ase.io import read, write
from ase.build import molecule, add_adsorbate
import numpy as np

def add_adsorbate_to_slab(slab, adsorbate, height, site='ontop'):
    """
    Add an adsorbate to a slab at a specified adsorption site.

    Parameters:
    - slab: ASE Atoms object, the pre-built BCC(111) slab
    - adsorbate: str, e.g., 'NH3', 'CO'
    - height: float, height above surface in Å
    - site: str, one of 'ontop', 'bridge', 'hollow'

    Returns:
    - ASE Atoms object with adsorbate added
    """
    # Get surface atoms (highest z)
    zmax = max(atom.position[2] for atom in slab)
    surface_atoms = [atom for atom in slab if abs(atom.position[2] - zmax) < 0.1]

    if len(surface_atoms) < 1:
        raise ValueError("No surface atoms found on slab.")

    # Determine site coordinates
    if site == 'ontop':
        x, y, _ = surface_atoms[0].position

    elif site == 'bridge':
        if len(surface_atoms) < 2:
            raise ValueError("Not enough atoms for bridge site.")
        a1, a2 = surface_atoms[0], surface_atoms[1]
        x = (a1.x + a2.x) / 2
        y = (a1.y + a2.y) / 2

    elif site == 'hollow':
        if len(surface_atoms) < 3:
            raise ValueError("Not enough atoms for hollow site.")
        x_center = sum(atom.x for atom in surface_atoms) / len(surface_atoms)
        y_center = sum(atom.y for atom in surface_atoms) / len(surface_atoms)

        def distance(atom, x, y):
            return np.sqrt((atom.x - x)**2 + (atom.y - y)**2)

        sorted_atoms = sorted(surface_atoms, key=lambda atom: distance(atom, x_center, y_center))
        a1, a2, a3 = sorted_atoms[:3]
        x = (a1.x + a2.x + a3.x) / 3
        y = (a1.y + a2.y + a3.y) / 3

    else:
        raise ValueError("site must be 'ontop', 'bridge', or 'hollow'")

    # Prepare adsorbate molecule
    mol = molecule(adsorbate)
    mol.rotate(180, 'x', center='COP')

    # Add adsorbate to slab
    add_adsorbate(slab, mol, height, position=(x, y))

    return slab

### Below code takes an optimized slab and adds the adsorbate using the above function. The structure is then geometrically optimized

In [ ]:
slab = read("your_slab_file.cif")  #  replace with your actual slab CIF file path
sites = ['ontop', 'bridge', 'hollow']
adsorbate = "NH3"
height = 3.0  # Angstroms

# ---- LOOP OVER TEST SITES ----
for site in sites:
    slab_copy = slab.copy()

    try:
        slab_with_ads = add_adsorbate_to_slab(slab_copy, adsorbate, height, site)
        print(f"✅ Adsorbate added successfully at {site} site.")

        # Save to CIF for inspection
        write(f"adsorbed_{adsorbate}_{site}.cif", slab_with_ads)

        # Run relaxation
        Calc_type = "relax"
        calc_label = f"{Element}_(3,3,4)_{adsorbate}_{site}_{Calc_type}"
        print(f"\n Starting relaxation for site: {site}")

        start = time.time()
        E = run_qe_calculation(slab_with_ads, Calc_type, pseudo_dict, calc_label)
        end = time.time()

        # Calculate duration
        duration = end - start
        hours = duration // 3600
        minutes = (duration % 3600) // 60
        seconds = duration
        remaining_seconds = seconds % 60

        print(f"✅ Relaxation finished for {site} site in", hours, "hours", minutes, "minutes,", remaining_seconds, "seconds")
        print(f"🧪 Energy of the {site} adsorbed surface: {E} eV")

    except Exception as e:
        print(f"❌ Failed at {site} site: {e}")